<a href="https://colab.research.google.com/github/Satyake/Open-AI-LLM-Langchain-Projects/blob/main/LLAMA_fine_tunning_LORA_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q datasets
!huggingface-cli login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.6 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/to

In [ ]:
from datasets import load_dataset
import re
dataset=load_dataset('timdettmers/openassistant-guanaco')
#shuffle dataset and slice
dataset=dataset['train'].shuffle(seed=42).select(range(1000))
def transform_conversation(example):
  conversation_text=example['text']
  segments=conversation_text.split('###')

  reformatted_segments=[]
  for i in range(1, len(segments)-1,2):
    human_text=segments[i].strip().replace('Human','').strip()
    if i+1<len(segments):
      assistant_text=segments[i+1].strip().replace('Assistant','').strip()
      reformatted_segments.append(f'<s>[INST] {human_text} [\INST] {assistant_text} </s>')
    else:
      reformatted_segments.append(f'<s>[INST] {human_text} [/INST]</s>')
  return {'text': ''.join(reformatted_segments)}

transformed_dataset=dataset.map(transform_conversation)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [1]:
#transformed_dataset['text']

In [ ]:
!pip -q install accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
model_name='NousResearch/Llama-2-7b-chat-hf'
new_model_name='Llama-2-7b-chat-new-finetuned'
###################
##LORA Parameters
###################
lora_r=64
lora_alpha=16
lora_dropout=0.1
##############################
## Bits and Bytes Parameters
###############################
use_4bit=True
bnb_4_bit_compute_dtype="float16"
#### fp4 or nf4 quantization type
bnb_4bit_quant_type="nf4"

#activate nested quantization for 4 bit base models (double quantization)
use_nested_quant=False

output_dir='./results'
num_train_epochs=1

fp16=False
bf16=False
per_device_train_batch_size=4
per_device_eval_batch_size=1
gradient_accumulation_steps=1
gradient_checkpointing=True
max_grad_norm=0.3 #gradient clipping
learning_rate=2e-4
weight_decay=0.001
optim='paged_adamw_32bit'
lr_scheduler_type='cosine'
max_steps=-1
#ratio of steps for a linear warmup
warmup_ratio=0.03
#group sequences into batches with same length
#saves memory and speeds up training
group_by_length=True
save_steps=0
#log every x updates steps
logging_steps=25

#SFT Parameters
max_seq_length=None
#pack multiple short examples in the same input sequence to increase efficiency
packing=False
#load model on GPU:0
device_map={"":0}


In [ ]:
transformed_dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [ ]:
#Load tokenizer and model with LORA configuration
compute_dtype=getattr(torch, bnb_4_bit_compute_dtype)


In [ ]:
bnb_config=BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_dtype=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant
)

In [ ]:
#load llama model
model=AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache=False
model.config.pretraining_tp=1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
#load llama tokenizer
tokenizer=AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right" #Fix weird overflow issue with fp16 training

In [ ]:
peft_config=LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias='none',
    task_type='CAUSAL_LM'
)

In [ ]:
training_arguments=TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to='tensorboard'
)

In [ ]:
trainer=SFTTrainer(
    model=model,
    train_dataset=transformed_dataset,
    peft_config=peft_config,
    dataset_text_field='text',
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing
)

In [ ]:
trainer.train()

In [ ]:
trainer.model.save_pretrained(new_model_name)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir results/runs


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-crede

In [ ]:
model=AutoModelForCausalLM.from_pretrained

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): 

In [ ]:
prompt="what is large language model?"
pipe=pipeline(task='text-generation', model=model, tokenizer=tokenizer, max_length=200)
result=pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] what is large language model? [/INST]  A large language model (LLM) is a type of artificial intelligence (AI) model that is trained on a large dataset of text, such as a corpus of books or a collection of web pages. The goal of an LLM is to learn the patterns and structures of language, and to be able to generate text that is coherent and natural-sounding.

LLMs are typically trained using a technique called "masked language modeling," in which some of the words in a given text are randomly replaced with a special token, and the model is trained to predict the original word that was replaced. This allows the model to learn the relationships between words in a language, and to generate text that is grammatically correct and semantically meaningful.

Some examples of LLMs include:

* BERT (Bidirectional Encoder Representations from Transformers


In [ ]:
#merge model with actual
base_model=AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map
)


model=PeftModel.from_pretrained(base_model,new_model_name)
model=model.merge_and_unload()

#reload tokenizer to save it

tokenizer=AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_size='right'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#empty vram
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()


NameError: name 'model' is not defined

In [ ]:
import locale
locale.getpreferredencoding= lambda: "UTF-8"
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-crede

In [ ]:
model.push_to_hub("Satyake/Llama-2-7b-chat-finetuned", check_pr=True)


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-66044da5-5f22f3b970284b2c2c5fbdce;0f7e63f1-5266-4e61-915b-84844f8e257e)

Repository Not Found for url: https://huggingface.co/api/models/Sayuake/Llama-2-7b-chat-finetuned_tokenizer.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.

In [ ]:
tokenizer.push_to_hub("Satyake/Llama-2-7b-chat-finetuned_tokenizer", check_pr=True)

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Satyake/Llama-2-7b-chat-finetuned_tokenizer/commit/3db0dfe111be535e2b0326c88c8bae359f678c4d', commit_message='Upload tokenizer', commit_description='', oid='3db0dfe111be535e2b0326c88c8bae359f678c4d', pr_url=None, pr_revision=None, pr_num=None)